## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hithadhoo,MV,-0.60,73.08,83.55,74,0,8.97,clear sky
1,1,Kapaa,US,22.08,-159.32,66.20,88,40,6.93,scattered clouds
2,2,Chichli,IN,22.83,78.82,70.30,43,0,4.94,clear sky
3,3,Ushuaia,AR,-54.80,-68.30,48.20,76,75,29.97,broken clouds
4,4,Bengkulu,ID,-3.80,102.27,76.51,87,100,4.27,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Chichli,IN,22.83,78.82,70.30,43,0,4.94,clear sky
4,4,Bengkulu,ID,-3.80,102.27,76.51,87,100,4.27,light rain
6,6,Arraial Do Cabo,BR,-22.97,-42.03,78.80,83,75,14.99,broken clouds
8,8,Beloha,MG,-25.17,45.05,73.26,57,0,16.67,clear sky
10,10,Rikitea,PF,-23.12,-134.97,75.13,70,66,5.75,broken clouds
30,30,Atuona,PF,-9.80,-139.03,77.34,73,30,12.97,scattered clouds
31,31,Alofi,NU,-19.06,-169.92,71.60,94,58,11.41,light rain
39,39,Hambantota,LK,6.12,81.12,75.20,100,90,2.24,overcast clouds
46,46,Victoria,HK,22.29,114.16,75.00,77,56,1.01,broken clouds
51,51,Hilo,US,19.73,-155.09,75.20,88,90,3.36,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID        116
City           116
Country        116
Lat            116
Lng            116
Max Temp       116
Humidity       116
Cloudiness     116
Wind Speed     116
Description    116
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Chichli,IN,70.30,clear sky,22.83,78.82,
4,Bengkulu,ID,76.51,light rain,-3.80,102.27,
6,Arraial Do Cabo,BR,78.80,broken clouds,-22.97,-42.03,
8,Beloha,MG,73.26,clear sky,-25.17,45.05,
10,Rikitea,PF,75.13,broken clouds,-23.12,-134.97,
30,Atuona,PF,77.34,scattered clouds,-9.80,-139.03,
31,Alofi,NU,71.60,light rain,-19.06,-169.92,
39,Hambantota,LK,75.20,overcast clouds,6.12,81.12,
46,Victoria,HK,75.00,broken clouds,22.29,114.16,
51,Hilo,US,75.20,overcast clouds,19.73,-155.09,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Country</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))